In [34]:
import time
import pandas as pd
import csv
#====== selenium activate esc key ======
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key

#====== selenium cathc NoSuchElementException ======
from selenium.common.exceptions import NoSuchElementException   
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

def get_selenium_param(filename):
    paramList = []
    with open(filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)

    return paramList

In [35]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)

In [43]:
def print_csv(filename):
    # 開啟 CSV 檔案
    with open('google_review/'+filename+'.csv', newline='') as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile)
        # 以迴圈輸出每一列
        for row in rows:
            print(row)

# Scrape url

In [84]:
#====== step1: get selenium parameter ======
store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)


#===== step2: scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chrome_driver/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index])# input store name
    time.sleep(5)
    
    # check if store name exist
    if check_exists_by_xpath('//a[@' + store_page_xpath[store_index] + ']'): # check if store name exist
        driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
        time.sleep(5)
    else:
        print(str(store_names[store_index])+' not exist......')
        driver.find_element_by_xpath('//input[@type="text"]').clear()
        continue
    
    
    # check if first page link exist
    if check_exists_by_xpath('//a[@' + first_page_xpath[store_index] + ']'):
        driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
        time.sleep(3)
    else:
        print(str(store_names[store_index]) + 'first page link not exist......')
        continue
        

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    
    # check if time of first page link exist, wait until get...
    for _ in range(10):
        if check_exists_by_xpath('//time[@class="_1o9PC Nzb55"]'):
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            time.sleep(3)
            break
        else:
            print(str(store_names[store_index]) + ' time not exist......' )
    

    # get 500 page url
    for i in range(6000):
        # check if reach end post, yes then break and transfer to next store
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        
        # check if time exist, wait until get...
        for _ in range(20):
            if check_exists_by_xpath('//time[@class="_1o9PC Nzb55"]'):
                post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
                stopDate = post_date[-1][0]
                break
            else:
                print(str(store_names[store_index]) + ' time not exist......' )
                time.sleep(3)

        
        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2018' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '12':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

['href="/p/CADW7fngF2Z/"', 'href="/p/CAF4DeEpl2a/"', 'href="/p/CACjRB0BU8c/"', 'href="/p/CACR8_3nQfc/"', 'href="/p/B_EdeflHCmg/"', 'href="/p/B_-G59hgggp/"', 'href="/p/CAC7cc5nhmt/"', 'href="/p/CAC5fgqn-hS/"', 'href="/p/B__ZLz3p-zZ/"', 'href="/p/CAFhbnvp9qN/"']
起家雞 Cheogajipfinished!
CHIMAC175 TAIPEIfinished!
KATZ 卡司 time not exist......
KATZ 卡司finished!
朴大哥的韓式炸雞 time not exist......
朴大哥的韓式炸雞 time not exist......
朴大哥的韓式炸雞finished!
努娜炸雞 누나치킨 - 逢甲商圈 time not exist......
努娜炸雞 누나치킨 - 逢甲商圈finished!
娘子炸雞finished!
Oppadak 歐巴答韓式炸雞 time not exist......
Oppadak 歐巴答韓式炸雞finished!
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak 韓式特色料理 time not exist......
TaeBak

# Scrape google map comment

In [37]:
#====== write into csv ======
def write_into_csv(filename, mydivs):
    counter = 0
    google_data = []
    # 開啟輸出的 CSV 檔案
    with open('google_review/'+filename+'.csv', 'w', newline='') as csvfile:
        # 建立 CSV 檔寫入器
        writer = csv.writer(csvfile)
        # 寫入column name
        writer.writerow(['name','detail','star','text','date'])

        # write review data
        for a in mydivs:
#             print(counter)
#             counter += 1
            if (a.find("div", class_="section-review-subtitle") != None) and (a.find("span", class_="section-review-text") != None):
                name = str(a.find("div", class_="section-review-title").text)
                reviewer_detail = str(a.find("div", class_="section-review-subtitle").text)
                review_stars = str(a.find("span", class_="section-review-stars").get('aria-label'))
                review_text = str(a.find("span", class_="section-review-text").text)
                review_date = a.find("span", class_="section-review-publish-date").text
                writer.writerow([name, reviewer_detail, review_stars, review_text, review_date])
    #         google_data.append([name, reviewer_detail, review_stars, review_text, review_date])



    # print(google_data)

In [75]:
#====== get url to scrape comments ======
assigned_url_list = []
assigned_star_list = []
google_store_name = []

with open('google_map_store_url', "r") as f:
    for line in f:
        tmp_list = line.split('\\')
        assigned_star_list.append(tmp_list[0])
        google_store_name.append(tmp_list[1])
        assigned_url_list.append(tmp_list[2].strip())

print(assigned_star_list)
print(google_store_name)
print(assigned_url_list)

['5']
['JAI 宅']
['https://www.google.it/maps/place/JAI+%E5%AE%85/@24.1488067,120.6811265,17z/data=!3m1!4b1!4m10!1m2!2m1!1zSkFJIOWuhQ!3m6!1s0x34693d692cdbdad9:0x6fc44f83b3a79fd7!8m2!3d24.1488067!4d120.6833152!9m1!1b1']


In [76]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import io
# from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains

#====== get all webpage html ======
for url_index,assigned_url in enumerate(assigned_url_list):
    driver = webdriver.Chrome(executable_path="chrome_driver/chromedriver") # Use Chrome
    driver.get(assigned_url) # open ig home page
    
    
    # select one star or five star review
    select_one_star = False
    select_five_star = False
    if assigned_star_list[url_index] == '1':
        select_one_star = True
    elif assigned_star_list[url_index] == '5':
        select_five_star = True
    
    '''scrape one star review''' 
    if (select_one_star):
        time.sleep(2)
        driver.find_element_by_xpath("//div[@class='cYrDcjyGO77__container']").click()
        time.sleep(2)
        driver.find_element_by_xpath("//div[@id='hovercard']//li[4]").click()
        
    '''scrape five star review'''
    if (select_five_star):
        time.sleep(2)
        driver.find_element_by_xpath("//div[@class='cYrDcjyGO77__container']").click()
        time.sleep(2)
        driver.find_element_by_xpath("//div[@id='hovercard']//li[3]").click()
        
    
    # scroll down to load more review
    stop_cnt = 0 # count to stop scroll down loop
    # len always get 0, so use stop_cnt to stop loop
    reviewCount = len(driver.find_elements_by_xpath("//div[@class='section-review ripple-container']"))
    time.sleep(10)
    # loading a minimum of 50 reviews
    while reviewCount <50: # change this number based on your requirement
        # load the reviews
        driver.find_element_by_xpath("//div[contains(@class,'section-loading-spinner')]").location_once_scrolled_into_view 
        # wait for loading the reviews
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='section-loading-overlay-spinner'][@style='display:none']")))
        # get the reviewsCount
        reviewCount = len(driver.find_elements_by_xpath("//div[@class='section-review ripple-container']"))
        stop_cnt += 1
#         print(stop_cnt)
        '''change number based on requirement, 1000: get 158 comments'''
        if stop_cnt == 2000:  
            break


    actions = ActionChains(driver)

    time.sleep(3)
#     content = driver.find_element_by_class_name('scrollable-show').click()
    htmlstring = driver.page_source
    afterstring=""
#     for _ in range(10000):
#         afterstring = htmlstring
#         actions.send_keys(Keys.PAGE_DOWN).perform()
#         htmlstring = driver.page_source
#         if afterstring == htmlstring:
#             print ("ended scraping crack test one")
#             actions.send_keys(Keys.PAGE_DOWN).perform()
#             htmlstring = driver.page_source
#             if afterstring == htmlstring:
#                 print("--Scrapping End--")
#                 break
#         time.sleep(3)

    #====== parse html ======
    print('start parsing html...')
    soup = BeautifulSoup(htmlstring,"lxml")
    mydivs = soup.findAll("div", {"class": "section-review-content"})
    
    # modify output filename depend on 5 star or 1 star
    if select_five_star:
        output_file_name = google_store_name[url_index]+'_5star'
        write_into_csv(output_file_name, mydivs)
    elif select_one_star:
        output_file_name = google_store_name[url_index]+'_1star'
        write_into_csv(google_store_name[url_index]+'_1star', mydivs)
    else:
        output_file_name = google_store_name[url_index]
        write_into_csv(google_store_name[url_index], mydivs)
    
    print(str(output_file_name)+' finished!')
    print ("Total reviews scraped:"+str(len(mydivs)))
    print('\n========================================\n')
    
        
    driver.close()

start parsing html...
JAI 宅_5star finished!
Total reviews scraped:1390




In [66]:
print_csv('辛殿麻辣鍋_5star')

['name', 'detail', 'star', 'text', 'date']
[' 幽幽 ', ' 在地嚮導 4 則評論 ', ' 5 顆星 ', '食材好吃！尤其牛舌獨特的Q彈口感讓整桌吃能牛的都讚不絕口(推薦涮七八分熟即可，別在鍋裡滾到老)，肉片類表現也不錯！蝦子、魷魚、蛤蠣等海鮮類也蠻鮮甜的。\n鴛鴦鍋，怕吃辣的朋友也有養生湯底能一同用餐。\n除了常見自助的冰淇淋、可樂等氣泡飲料外，菜單上也有點心可供選擇，劃好由店家為您送到桌邊。有布丁、奶酪、千層蛋糕、彩虹起司蛋糕(大概六、七種吧名稱可能有記錯)。 …', '3 週前']
[' Catherine ', ' 在地嚮導 12 則評論 ', ' 5 顆星 ', '4/7中午來用餐，店員態度很好（又帥），有求必應，主動觀察、詢問客人是否有需要協助的地方，店內沒供應的東西但客人提出也會想辦法弄給我們，食材豐富、東西好吃～餐飲業如果都能跟他們一樣就好～', '1 個月前']
[' 在地嚮導pro ', ' 在地嚮導 ・42 則評論 ', ' 5 顆星 ', '好吃的吃到飽火鍋\n會提供白鍋跟紅鍋\n超級推薦牛小排👍\n白蝦也非常新鮮👍\n奶酪不輸店面貨👍\n整體來說五顆星👍', '1 個月前']
[' LIYING YANG ', ' 在地嚮導 ・118 則評論 ', ' 5 顆星 ', '訂了晚上20:00的位置，準時到場後店員很客氣的請我們在外頭稍後，很幸運的坐到沙發區的位置。一點完菜過沒多久都八盤八盤的送過來，上菜速度飛快！我們掃幾盤他們就收幾盤！店員多、服務效率佳！\n\n麻辣鍋底的鴨血和豆腐是可以請店員補的喔！湯只要少於一半店員就會快速過來補湯底，服務真的非常有效率！而我們自己在確認什麼菜或肉沒上到時，店員聽見也前來關心是否有遺漏出餐的地方。 …', '4 個月前']
[' chen yu Hsieh ', ' 在地嚮導 3 則評論 ', ' 5 顆星 ', '上菜速度快，肉也切滿厚，冰淇淋有哈根達斯和莫凡彼，真要挑剔可能就是麻辣鍋不夠辣，玉米筍要自己剝皮，但平日午餐一人592我覺得算不錯了', '1 個月前']
[' Jhe-Han Ko ', ' 在地嚮導 11 則評論 ', ' 5 顆星 ', '常定不到位子，但服務不錯，東西也好吃', '3 週前']
[' 溫玉娟 ', ' 在地嚮導 32 則

In [ ]:
    with open(filename+'.csv', newline='') as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile)
        # 以迴圈輸出每一列
        for row in rows:
            print(row)

In [211]:
#====== word segment ======
import os
for filename in os.listdir('google_review/'):
    with open('google_review/'+filename, newline='') as f: # open in readonly mode
        rows = csv.reader(f)
        
        # 以迴圈輸出每一列
        firstline = True
        for row in rows:
            if firstline == True:
                firstline = False
                continue
            print(row)

[' 狐狸狗狗 ', ' 在地嚮導 ・147 則評論 ', ' 5 顆星 ', '很有特色的一間店，佈置十分特殊，餐點也很可口！\n晚上不知道要吃甚麼，開著車也不知道要落腳何處，終於挑選到這間地下室可以停車的餐廳，剛到的時候感覺這不是商務人士的餐廳，但是又不想再繼續尋找，於是就抱著嘗鮮的心態將要開啟門，沒想到一開始的門就令我吃驚，因為壓了壓竟然沒反應，試了很多方式才發現，原來是要推開，那上面長長的開門提示作用是....... …', '6 天前']
[' Yi Small ', ' 在地嚮導 3 則評論 ', ' 4 顆星 ', '店裡的氛圍跟裝潢都很吸引人，不過是情侶聚餐、朋友聚會都很適合。服務員也會細心介紹餐點及特色，餐點真的多到讓人有選擇障礙，不過也讓人下次會想再度光臨，實在想吃的太多，但是胃容量有限，剛好提供下次來的藉口（笑）\n跟一般店不一樣，他們不是收一成服務費，而是一組客人收20元，這點對荷包蠻友善的，尤其現在餐廳林立，單價普遍都偏高的狀態，JAI宅的餐點cp我覺得蠻高的。 …', '1 週前']
[' 姚俐安 ', ' 在地嚮導 ・87 則評論 ', ' 4 顆星 ', '來台中這麼多次終於第一次來\n櫃檯的女生還有帶位的女生都很親切服務態度很好\n店內環境漂亮但是燈光沒有很適合拍照\n如果拍照應該要調色才會好看\n餐點免費加飯加麵\n但是加了的量沒有很多\n點蛤蜊燉飯你會挑飯挑到瘋掉\n殼裡面會卡很多飯\n醬汁沒有很多 飯會黏死死的\n因為份量已經不多了\n要是不吃殼裡面的飯根本吃不飽：）\n少的一顆星是送餐的男生 有戴帽子有耳環\n呃印象中還有戴眼鏡跟刺青\n送餐碗放超大力\n我把餐點明細放在靠近我的位置\n在我手機旁邊 胸口前面的桌子上\n他真的是一句話都沒說 …', '1 個月前']
[' 曾小亭 ', ' 在地嚮導 ・260 則評論 ', ' 4 顆星 ', '黑松露板腱牛肉\n牛肉熟度很恰好，很軟嫩。\n唯一最推薦～\n\n奶香清醬炭烤照燒鮭魚義大利麵\n唯一第二推薦～\n\n可能因疫情關係，出餐速度太過於快，快到大夥們消化不了，出餐速度沒掌控好，主餐還沒吃完，副餐一直送然後也冷了，也沒那麼美味。\n\n泰式雞翅真的無敵soso覺得沒味\n披薩也沒有驚艷就很普通不要有太大期待。', '1 週前']
[' 張仕哲 ', ' 在地嚮導 2 則

[' 黃柏鈞 ', ' 在地嚮導 ・211 則評論 ', ' 3 顆星 ', '除了價格貴了些  其餘忠於原味（請選濃郁湯頭）', '7 個月前']
[' Sunghsun Lin ', ' 在地嚮導 ・124 則評論 ', ' 5 顆星 ', '第一次吃的人一定要選標準加一點辣，選淡味的味道不夠不好吃。標準不會太鹹', '7 個月前']
[' 廖小章 ', ' 在地嚮導 ・121 則評論 ', ' 1 顆星 ', '食物不錯，但是服務人員素質不佳，且等太久，結帳時一定要先去上廁所再結帳，要不然會被誤會沒付錢，引起誤會，態度非常差，希望人員素質可以提高。', '7 個月前']
[' CASTLE LIN ', ' 在地嚮導 ・236 則評論 ', ' 4 顆星 ', '過新年，吃一蘭，福氣滿籃', '3 個月前']
[' Na Ha ', ' 在地嚮導 ・34 則評論 ', ' 5 顆星 ', '吃了以後，嘴巴被養刁了😂\n現在都不會在想吃別的拉麵了，\n喜歡一蘭拉麵的麵條口感❤️\n可以自己選擇加什麼料，多油，多鹹，多辣，\n而且服務態度非常好!送完餐點都會鞠躬在關簾子，覺得非常有禮貌', '7 個月前']
[' 夏大寶 ', ' 在地嚮導 ・52 則評論 ', ' 3 顆星 ', '適合台灣人的口味，但真的不建議排隊等太久。同樣的價位，還有更多有趣的拉麵等你發掘（但如果你不想踩雷，這也是你不錯選擇）', '7 個月前']
[' Elvis Fu ', ' 在地嚮導 11 則評論 ', ' 1 顆星 ', '一蘭拉麵一直是我的最愛，從日本到台灣，各地加總消費次數至少數十次。但是不知道為什麼， …', '11 個月前']
[' 蘇晉賢 ', ' 在地嚮導 1 則評論 ', ' 1 顆星 ', '叫人領號碼牌然後前面後等100多組要人看線上後等進度快到再過來\n結果還沒走到新光三越就瞬間跳50組走回去就看到超多人在排隊\n給了號碼牌也是叫人重排\n那到底抽號碼牌幹嘛？\n就一開始讓人排隊不就好🙄🙄🙄', '7 個月前']
[' steven Huang ', ' 在地嚮導 ・94 則評論 ', ' 2 顆星 ', '唉，懷著朝的心情來吃，因為之前在日本九州吃過之後就念念不忘........結論是不推薦來吃因爲又貴又少吃不飽。比日本的差服務的態度普普，要吃的話還是去日本

In [222]:
review_star = int(row[2].strip()[0])
review_text = row[3].strip()
print(review_text)

環境很乾淨，拉麵是臺灣很很喜歡的豚骨湯頭，麵體是標準九州拉麵那種細麵體，整體來說算是好吃，但真的沒有好吃到要排超過半小時。路過沒人可以吃。


In [142]:
#====== create google review dataframe ======
google_review_df = pd.DataFrame(google_data,columns=['name','detail','star','text','date'])
google_review_df
# google_review_df[google_review_df['star'] == ' 5 顆星 ']

,name,detail,star,text,date
0,狐狸狗狗,在地嚮導 ・147 則評論,5 顆星,很有特色的一間店，佈置十分特殊，餐點也很可口！\n晚上不知道要吃甚麼，開著車也不知道要落腳何...,5 天前
1,Yi Small,在地嚮導 3 則評論,4 顆星,店裡的氛圍跟裝潢都很吸引人，不過是情侶聚餐、朋友聚會都很適合。服務員也會細心介紹餐點及特色，...,1 週前
2,姚俐安,在地嚮導 ・87 則評論,4 顆星,來台中這麼多次終於第一次來\n櫃檯的女生還有帶位的女生都很親切服務態度很好\n店內環境漂亮但...,1 個月前
3,曾小亭,在地嚮導 ・258 則評論,4 顆星,黑松露板腱牛肉\n牛肉熟度很恰好，很軟嫩。\n唯一最推薦～\n\n奶香清醬炭烤照燒鮭魚義大利...,1 週前
4,張仕哲,在地嚮導 2 則評論,5 顆星,一段時間沒來，擺飾又換了風格，這次像是在森林裡用餐，非常放鬆，義大利麵一樣美味，十分舒適。,3 天前
...,...,...,...,...,...
113,歐有容,在地嚮導 ・39 則評論,5 顆星,東西好吃又好拍，可以選擇要不要付服務費20摳。,3 個月前
114,Hsu Sharon,在地嚮導 ・23 則評論,5 顆星,喜歡橘醬 超好吃 料也很多\n餐廳環境裝潢優美好拍照\n店員態度佳,3 個月前
115,Po-Hsiang Lai,在地嚮導 ・5 則評論,5 顆星,CP值高 適合朋友聚聚 儘管是食量大的人也可以很滿足,3 個月前
116,蔡佳孚,在地嚮導 ・44 則評論,4 顆星,蝦子口味的餐點都有剝殼又很多隻😋 太貼心了👍,2 個月前


In [ ]:
#====== write into txt ======
textdoc = io.open("gmapreview.txt", "w", encoding="utf-8")
soup = BeautifulSoup(htmlstring,"lxml")
mydivs = soup.findAll("div", {"class": "section-review-content"})
print(len(mydivs))

counter = 0
for a in mydivs: 
    print(counter)
    textdoc.write(str("\nReviewer name: "+a.find("div", class_="section-review-title").text)+" \n||Reviewer Detail: " + str(a.find("div", class_="section-review-subtitle").text) +" \n||Reviewerer Profile URL:"+ str(a.find("a").get('href')))
    textdoc.write(str("\n"+a.find("span", class_="section-review-stars").get('aria-label')))
    textdoc.write(" \n||" + a.find("span", class_="section-review-text").text+" \n|| " + a.find("span", class_="section-review-publish-date").text)
    textdoc.write("=========================================\n")
    counter = counter + 1
print ("Total reviews scraped:"+str(counter))
textdoc.close()

In [27]:
with open("htmlstring_google_comment.txt", "w") as text_file:
    text_file.write(htmlstring)